In [1]:
import numpy as np
import math
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
class DecisionStump():
    def __init__(self,feature_index=None,threshold=None,alpha=None,polarity=1):
        self.feature_index=feature_index
        self.threshold=threshold
        self.alpha=alpha
        self.polarity=polarity
class AdaboostClassifier():
    def __init__(self,n_clifs=50):
        self.n_clifs=n_clifs
    def fit(self,X,y):
        samples,features=np.shape(X)
        self.clfs=[]
        w=np.full(samples,(1/samples))
        for i in range(self.n_clifs):
            clf=DecisionStump()
            min_error=float("inf")
            for feature in range(features):
                unique_values=np.unique(np.expand_dims(X[:,feature],axis=1))
                for unique in unique_values:
                    p=1
                    predictions=np.ones(np.shape(y))
                    predictions[X[:,feature]<unique]=-1
                    error=sum(w[y!=predictions])
                    if(error>0.5):
                        error=1-error
                        p=-1
                    if(error<min_error):
                        clf.feature_index=feature
                        clf.threshold=unique
                        clf.polarity=p
                        min_error=error
            clf.alpha=0.5*math.log((1-min_error)/(min_error+1e-10))
            predictions=np.ones(np.shape(y))
            negative_index=(clf.polarity * X[:,clf.feature_index]<clf.polarity*clf.threshold)
            predictions[negative_index]=-1
            w *=np.exp(-clf.polarity*y*predictions)
            w /=np.sum(w)
            self.clfs.append(clf)
    def predict(self,X):
        samples=np.shape(X)[0]
        preds=np.zeros((samples,1))
        for clf in self.clfs:
            predictions=np.ones(np.shape(preds))
            negative_index=(clf.polarity*X[:,clf.feature_index]<clf.polarity*clf.threshold)
            predictions[negative_index]=-1
            preds+=clf.alpha*predictions
        preds=np.sign(preds).flatten()
        return preds
            
            
        
            
                        
                
                
        

In [3]:
from sklearn.datasets import load_breast_cancer
X=load_breast_cancer().data
y=load_breast_cancer().target
y[y==0]=-1

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
ad=AdaboostClassifier()
ad.fit(X_train,y_train)

In [6]:
pred=ad.predict(X_test)

In [7]:
print(accuracy_score(y_test,pred))

0.8947368421052632
